<a href="https://colab.research.google.com/github/faiyazansariusa/ColabAI/blob/main/Model_finetune_createmodel_in_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install --upgrade datasets fsspec
from datasets import load_dataset
imdb = load_dataset("imdb")

In [4]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
import numpy as np
from evaluate import load

def compute_metrics(eval_pred):
   load_accuracy = load("accuracy")
   load_f1 = load("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [12]:
#!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [17]:
from huggingface_hub import notebook_login
notebook_login()

In [18]:
from transformers import TrainingArguments, Trainer
repo_name = "faiyaz_gen_ai_finetune"
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

/tmp/ipython-input-18-4142267244.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: faiyazansari-usa (faiyazansari-usa-synechron) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=376, training_loss=0.2887476860208714, metrics={'train_runtime': 496.4391, 'train_samples_per_second': 12.086, 'train_steps_per_second': 0.757, 'total_flos': 782725021021056.0, 'train_loss': 0.2887476860208714, 'epoch': 2.0})

In [20]:
trainer.evaluate()

{'eval_loss': 0.34202584624290466,
 'eval_accuracy': 0.8733333333333333,
 'eval_f1': 0.8766233766233766,
 'eval_runtime': 6.0387,
 'eval_samples_per_second': 49.68,
 'eval_steps_per_second': 3.146,
 'epoch': 2.0}

In [21]:
trainer.push_to_hub()

events.out.tfevents.1752732805.0e386991988f.2157.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/faiyazansariusa/faiyaz_gen_ai_finetune/commit/072b57d80226f5e101f44fe4048af97c1e08f492', commit_message='End of training', commit_description='', oid='072b57d80226f5e101f44fe4048af97c1e08f492', pr_url=None, repo_url=RepoUrl('https://huggingface.co/faiyazansariusa/faiyaz_gen_ai_finetune', endpoint='https://huggingface.co', repo_type='model', repo_id='faiyazansariusa/faiyaz_gen_ai_finetune'), pr_revision=None, pr_num=None)

In [22]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="faiyazansariusa/faiyaz_gen_ai_finetune")

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [23]:
pipe("I am happy today to learn ML")

[{'label': 'LABEL_1', 'score': 0.9535205960273743}]

In [26]:
pipe("I am not happy")

[{'label': 'LABEL_0', 'score': 0.8596178889274597}]

In [ ]:
#Assessment to load Excelsheet read review add column sentiment and update the status of sentiment with Positive and negative feedback

In [28]:
# prompt: Write a python code to read "/content/sentiment_test.xlsx".Read column Review and based on it do sentiment analysis using model="faiyazansariusa/faiyaz_gen_ai_finetune" . After doing text-classification update it status in column sentiment and i should be able to download the updated sentiment_test.xlsx file

!pip install openpyxl
!pip install pandas
import pandas as pd
from google.colab import files

# Read the Excel file
df = pd.read_excel("/content/sentiment_test.xlsx")

# Initialize an empty list to store sentiment predictions
sentiments = []

# Iterate through the 'Review' column and perform sentiment analysis
for review in df['Review']:
    # Use the sentiment analysis pipeline
    result = pipe(review)
    # Extract the predicted label (e.g., 'LABEL_0', 'LABEL_1')
    label = result[0]['label']
    # Map the label to 'Positive' or 'Negative' based on the model's output
    # You might need to adjust the mapping based on how the model was trained
    sentiment = 'Positive' if label == 'LABEL_1' else 'Negative'
    sentiments.append(sentiment)

# Add the sentiment predictions to a new column in the DataFrame
df['sentiment'] = sentiments

# Save the updated DataFrame to a new Excel file
output_filename = "sentiment_test_updated.xlsx"
df.to_excel(output_filename, index=False)

# Download the updated file
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
# prompt: Read the "/content/BankReviews.xlsx" and pass this data to  model="faiyazansariusa/faiyaz_gen_ai_finetune". Do Sentiment analysis based on column Stars in "/content/BankReviews.xlsx"
# Please predict this data and create a column Status with Sentiment analysis based on column Stars.The new sheet created should downloadable

# Read the Excel file
df = pd.read_excel("/content/BankReviews.xlsx")

# Initialize an empty list to store sentiment predictions
sentiments = []

# Use the pipeline defined in the preceding code, which is already loaded with the specified model
pipe = pipeline("text-classification", model="faiyazansariusa/faiyaz_gen_ai_finetune")

# Iterate through the rows and perform sentiment analysis based on the 'Stars' column
for index, row in df.iterrows():
    stars = row['Stars']
    # Assign sentiment based on the number of stars
    # Assuming 4 or 5 stars is Positive, and 1, 2, or 3 stars is Negative
    sentiment = 'Positive' if stars in [4, 5] else 'Negative'
    sentiments.append(sentiment)

# Add the sentiment predictions to a new column in the DataFrame
df['Status'] = sentiments

# Save the updated DataFrame to a new Excel file
output_filename = "BankReviews_sentiment_updated.xlsx"
df.to_excel(output_filename, index=False)

# Download the updated file
files.download(output_filename)

Device set to use cuda:0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>